# 4.1.3 物體追蹤 (Object Tracking)

**WBS 4.1.3**: 光流法與物體追蹤技術

## 學習目標
- 理解光流法 (Optical Flow) 的原理與應用
- 掌握 Lucas-Kanade 稀疏光流追蹤
- 學習 Farneback 稠密光流
- 實現多目標追蹤基礎
- (進階) 卡爾曼濾波器基礎
- 比較不同追蹤方法的性能

**難度等級**: ⭐⭐⭐⭐ (進階)  
**預估時間**: 120 分鐘  
**WBS編號**: 4.1.3

<div style="page-break-after: always"></div>

# 1. 光流法概述

## 1-1: 什麼是光流 (Optical Flow)?

> **光流** 是描述影像中像素移動模式的向量場，表示連續影格之間像素的運動方向和速度。

### 核心概念

```
光流法基本假設:

1. 亮度恆定假設 (Brightness Constancy)
   └─> 同一像素在連續影格中亮度不變
       I(x, y, t) = I(x+dx, y+dy, t+dt)

2. 小運動假設 (Small Motion)
   └─> 連續影格間的位移很小

3. 空間一致性假設 (Spatial Coherence)
   └─> 鄰近像素具有相似的運動
```

### 光流方程式

> **泰勒展開與光流約束方程**:
>
> $$I(x+dx, y+dy, t+dt) \approx I(x,y,t) + \frac{\partial I}{\partial x}dx + \frac{\partial I}{\partial y}dy + \frac{\partial I}{\partial t}dt$$
>
> 根據亮度恆定假設:
>
> $$\frac{\partial I}{\partial x}\frac{dx}{dt} + \frac{\partial I}{\partial y}\frac{dy}{dt} + \frac{\partial I}{\partial t} = 0$$
>
> 簡化為:
>
> $$I_x u + I_y v + I_t = 0$$
>
> 其中:
> * $I_x, I_y$ 是影像的空間梯度
> * $I_t$ 是影像的時間梯度
> * $(u, v)$ 是光流向量 (要求解)

### Aperture Problem

> **孔徑問題**: 一個方程式有兩個未知數 (u, v)，無法唯一求解！
>
> **解決方案**:
> * **稀疏光流**: 追蹤特定特徵點 (Lucas-Kanade)
> * **稠密光流**: 為所有像素計算光流 (Farneback, Horn-Schunck)

## 1-2: 光流法的分類

| 類型 | 方法 | 特點 | 應用 |
|------|------|------|------|
| **稀疏光流** | Lucas-Kanade | 追蹤少量特徵點 | 物體追蹤、視覺測程 |
| **稠密光流** | Farneback | 計算所有像素的光流 | 視頻穩定、運動分析 |
| **深度學習** | FlowNet, PWC-Net | 端到端學習 | 自動駕駛、動作識別 |

## 1-3: 光流法的應用

> * **物體追蹤**: 追蹤視頻中的目標物體
> * **運動估計**: 分析場景中的運動模式
> * **視頻壓縮**: MPEG 編碼中的運動補償
> * **動作識別**: 識別人體動作和手勢
> * **視頻穩定**: 相機抖動補償
> * **自動駕駛**: 場景理解和障礙物檢測

## 環境設置與導入

In [ ]:
import sys
import os

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Configure matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print(f'✅ OpenCV version: {cv2.__version__}')
print(f'✅ NumPy version: {np.__version__}')

# Check available video files
video_dir = '../assets/videos/'
if os.path.exists(video_dir):
    videos = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
    print(f'✅ Available videos: {len(videos)}')
    for v in videos:
        print(f'   - {v}')
else:
    print('⚠️ Video directory not found')

### 輔助函數：影像視覺化工具

In [ ]:
def draw_flow(img, flow, step=16):
    """
    Draw optical flow on image
    
    Args:
        img: Input image (BGR)
        flow: Optical flow (H x W x 2)
        step: Sampling step for visualization
    
    Returns:
        vis: Visualization image
    """
    h, w = img.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    
    # Create line endpoints
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    
    # Draw flow vectors
    vis = img.copy()
    for (x1, y1), (x2, y2) in lines:
        # Calculate magnitude for color coding
        mag = np.sqrt((x2-x1)**2 + (y2-y1)**2)
        
        # Color based on magnitude
        if mag > 2:  # Only draw significant motion
            color = (0, int(255 * min(mag/20, 1)), 0)
            cv2.arrowedLine(vis, (x1, y1), (x2, y2), color, 1, cv2.LINE_AA, tipLength=0.3)
    
    return vis


def draw_hsv_flow(flow):
    """
    Convert flow to HSV color representation
    
    Args:
        flow: Optical flow (H x W x 2)
    
    Returns:
        bgr: HSV flow visualization in BGR
    """
    h, w = flow.shape[:2]
    fx, fy = flow[:, :, 0], flow[:, :, 1]
    
    # Convert to polar coordinates
    mag, ang = cv2.cartToPolar(fx, fy)
    
    # Create HSV image
    hsv = np.zeros((h, w, 3), dtype=np.uint8)
    hsv[..., 0] = ang * 180 / np.pi / 2  # Hue: direction
    hsv[..., 1] = 255  # Saturation: full
    hsv[..., 2] = np.uint8(np.minimum(mag * 4, 255))  # Value: magnitude
    
    # Convert to BGR
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr


def create_color_wheel():
    """
    Create a color wheel legend for optical flow visualization
    """
    # Create circular color wheel
    size = 200
    wheel = np.zeros((size, size, 3), dtype=np.uint8)
    
    y, x = np.ogrid[-size//2:size//2, -size//2:size//2]
    radius = np.sqrt(x*x + y*y)
    angle = np.arctan2(y, x)
    
    # Create HSV wheel
    hsv = np.zeros((size, size, 3), dtype=np.uint8)
    hsv[..., 0] = ((angle + np.pi) / (2 * np.pi) * 180).astype(np.uint8)
    hsv[..., 1] = 255
    hsv[..., 2] = np.uint8(np.minimum(radius / (size//2) * 255, 255))
    
    # Mask circle
    mask = radius <= size // 2
    hsv[~mask] = 0
    
    wheel = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    # Add labels
    cv2.putText(wheel, 'Right', (size-50, size//2), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(wheel, 'Left', (5, size//2), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(wheel, 'Up', (size//2-10, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(wheel, 'Down', (size//2-20, size-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    return wheel

print('✅ Utility functions loaded')

<div style="page-break-after: always"></div>

# 2. Lucas-Kanade 光流法

## 2-1: Lucas-Kanade 原理

> **Lucas-Kanade** 是最經典的稀疏光流算法，由 Bruce D. Lucas 和 Takeo Kanade 於 1981 年提出。

### 核心思想

```
Lucas-Kanade 方法:

1. 假設局部鄰域內的像素有相同的運動
   └─> 將單一方程式問題轉為過定方程組

2. 在 n×n 窗口內建立方程組
   └─> 對於每個像素: I_x·u + I_y·v + I_t = 0

3. 使用最小平方法求解
   └─> 最小化誤差的平方和
```

### 數學推導

> **矩陣形式**:
>
> $$A \mathbf{v} = \mathbf{b}$$
>
> 其中:
>
> $$A = \begin{bmatrix} I_x(p_1) & I_y(p_1) \\ I_x(p_2) & I_y(p_2) \\ \vdots & \vdots \\ I_x(p_n) & I_y(p_n) \end{bmatrix}, \quad \mathbf{v} = \begin{bmatrix} u \\ v \end{bmatrix}, \quad \mathbf{b} = -\begin{bmatrix} I_t(p_1) \\ I_t(p_2) \\ \vdots \\ I_t(p_n) \end{bmatrix}$$
>
> **最小平方解**:
>
> $$\mathbf{v} = (A^T A)^{-1} A^T \mathbf{b}$$
>
> 展開為:
>
> $$\begin{bmatrix} u \\ v \end{bmatrix} = \begin{bmatrix} \sum I_x^2 & \sum I_x I_y \\ \sum I_x I_y & \sum I_y^2 \end{bmatrix}^{-1} \begin{bmatrix} -\sum I_x I_t \\ -\sum I_y I_t \end{bmatrix}$$

### 金字塔 Lucas-Kanade

> **問題**: 基本 LK 方法只適用於小位移
>
> **解決**: 使用**影像金字塔** (Image Pyramid) 處理大位移:
>
> ```
> 金字塔處理流程:
> 
> Level 2 (最小) ─┐
>                 ├─> 從粗糙到精細
> Level 1         │   逐層求解光流
>                 │
> Level 0 (原始) ─┘
> ```

## 2-2: OpenCV 實現

### cv2.calcOpticalFlowPyrLK() 函數

```python
nextPts, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, nextPts)
```

> **參數說明**:
> * `prevImg`: 前一幀灰度影像
> * `nextImg`: 當前幀灰度影像
> * `prevPts`: 前一幀的特徵點 (N×1×2 array)
> * `nextPts`: 當前幀的特徵點 (初始猜測，可為 None)
> * `winSize`: 搜索窗口大小 (預設 (21, 21))
> * `maxLevel`: 金字塔層數 (預設 3)
>
> **返回值**:
> * `nextPts`: 當前幀中的特徵點位置
> * `status`: 追蹤狀態 (1=成功, 0=失敗)
> * `err`: 追蹤誤差

## 2-3: Lucas-Kanade 實作 - 單點追蹤

In [ ]:
# Load video or create synthetic video
video_path = '../assets/videos/car_chase_01.mp4'

if os.path.exists(video_path):
    cap = cv2.VideoCapture(video_path)
    print(f'✅ Loaded video: {video_path}')
else:
    # Create synthetic moving ball video
    print('⚠️ Video not found, creating synthetic video')
    cap = None

# Read first frame
if cap is not None:
    ret, frame1 = cap.read()
    if not ret:
        print('❌ Failed to read video')
        cap = None

# If no video, create synthetic data
if cap is None:
    # Create synthetic video with moving circle
    frames = []
    for i in range(50):
        frame = np.ones((480, 640, 3), dtype=np.uint8) * 255
        x = int(100 + i * 10)
        y = int(240 + 50 * np.sin(i * 0.2))
        cv2.circle(frame, (x, y), 30, (0, 0, 255), -1)
        frames.append(frame)
    frame1 = frames[0]
    print('✅ Created synthetic video with moving circle')

# Convert to grayscale
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Detect good features to track (using Shi-Tomasi)
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7
)

p0 = cv2.goodFeaturesToTrack(gray1, mask=None, **feature_params)

# Display first frame with detected features
frame1_display = frame1.copy()
if p0 is not None:
    for i in p0:
        x, y = i.ravel()
        cv2.circle(frame1_display, (int(x), int(y)), 5, (0, 255, 0), -1)

plt.figure(figsize=(12, 6))
plt.imshow(cv2.cvtColor(frame1_display, cv2.COLOR_BGR2RGB))
plt.title(f'Initial Features to Track ({len(p0)} points)', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

print(f'\n✅ Lucas-Kanade 初始化:')
print('='*60)
print(f'初始特徵點數量: {len(p0)}')
print(f'影像尺寸: {frame1.shape[:2]}')

### Lucas-Kanade 追蹤實作

In [ ]:
# Parameters for Lucas-Kanade optical flow
lk_params = dict(
    winSize=(15, 15),          # Search window size
    maxLevel=2,                # Pyramid levels
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)

# Create random colors for tracking visualization
color = np.random.randint(0, 255, (len(p0), 3))

# Create mask for drawing
mask = np.zeros_like(frame1)

# Track through multiple frames
num_frames_to_track = 30
tracked_frames = []

# Prepare for tracking
if cap is not None:
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to first frame
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    frame_idx = 0
else:
    old_gray = gray1.copy()
    frame_idx = 0

p0_orig = p0.copy()

for i in range(num_frames_to_track):
    # Read next frame
    if cap is not None:
        ret, frame = cap.read()
        if not ret:
            break
    else:
        if frame_idx + 1 >= len(frames):
            break
        frame = frames[frame_idx + 1]
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    # Select good points
    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]
    else:
        break
    
    # Draw tracks
    frame_vis = frame.copy()
    for j, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        
        # Draw line on mask
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), 
                       color[j].tolist(), 2)
        
        # Draw circle on frame
        frame_vis = cv2.circle(frame_vis, (int(a), int(b)), 5, 
                              color[j].tolist(), -1)
    
    # Combine frame and mask
    output = cv2.add(frame_vis, mask)
    
    # Add frame info
    cv2.putText(output, f'Frame: {i+1}/{num_frames_to_track}', (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(output, f'Tracked: {len(good_new)} points', (10, 60),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    tracked_frames.append(output)
    
    # Update for next iteration
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
    frame_idx += 1

# Display tracking results (show every 5th frame)
if len(tracked_frames) > 0:
    display_indices = [0, len(tracked_frames)//4, len(tracked_frames)//2, 
                      3*len(tracked_frames)//4, -1]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    for idx, i in enumerate(display_indices):
        axes[idx].imshow(cv2.cvtColor(tracked_frames[i], cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f'Frame {i+1}', fontsize=12)
        axes[idx].axis('off')
    
    # Add color wheel legend
    wheel = create_color_wheel()
    axes[5].imshow(cv2.cvtColor(wheel, cv2.COLOR_BGR2RGB))
    axes[5].set_title('Motion Direction Legend', fontsize=12)
    axes[5].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print('\n✅ Lucas-Kanade 追蹤結果:')
    print('='*60)
    print(f'總追蹤幀數: {len(tracked_frames)}')
    print(f'初始特徵點: {len(p0_orig)}')
    print(f'最終追蹤點: {len(good_new)}')
    print(f'追蹤保留率: {len(good_new)/len(p0_orig)*100:.1f}%')

# Close video capture
if cap is not None:
    cap.release()

### Lucas-Kanade 參數影響分析

In [ ]:
# Compare different window sizes
print('\n📊 Lucas-Kanade 參數影響分析:')
print('='*60)

window_sizes = [(7, 7), (15, 15), (21, 21), (31, 31)]

# Reset to first two frames
if cap is None and 'frames' in locals():
    frame1 = frames[0]
    frame2 = frames[5]
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
else:
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    for _ in range(5):
        ret, frame2 = cap.read()
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    cap.release()

# Detect features
p0 = cv2.goodFeaturesToTrack(gray1, maxCorners=50, qualityLevel=0.3, 
                             minDistance=7, blockSize=7)

results = []

for win_size in window_sizes:
    lk_params_test = dict(
        winSize=win_size,
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
    )
    
    # Calculate flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(gray1, gray2, p0, None, **lk_params_test)
    
    if p1 is not None:
        good = p1[st == 1]
        success_rate = len(good) / len(p0) * 100
        avg_error = np.mean(err[st == 1]) if np.any(st == 1) else 0
    else:
        success_rate = 0
        avg_error = 0
    
    results.append({
        'window': win_size,
        'success_rate': success_rate,
        'avg_error': avg_error
    })
    
    print(f'Window Size {win_size}: {success_rate:.1f}% success, '
          f'avg error: {avg_error:.3f}')

print('\n💡 參數調整建議:')
print('   - winSize 小 (7x7): 快速但對大位移不穩定')
print('   - winSize 中 (15x15): 平衡速度與準確度 [推薦]')
print('   - winSize 大 (31x31): 穩定但計算慢，適合大位移')
print('   - maxLevel 高: 可處理更大的位移')
print('='*60)

<div style="page-break-after: always"></div>

# 3. Farneback 稠密光流

## 3-1: Farneback 原理

> **Farneback 光流** 是一種計算稠密光流的方法，由 Gunnar Farnebäck 於 2003 年提出。

### 核心思想

```
Farneback 方法:

1. 多項式展開 (Polynomial Expansion)
   └─> 在每個像素鄰域用二次多項式近似信號
       f(x) ≈ x^T A x + b^T x + c

2. 多項式係數比較
   └─> 比較兩幀中相同位置的多項式係數

3. 全局位移場
   └─> 為所有像素計算光流向量
```

### Farneback vs Lucas-Kanade

| 特性 | Lucas-Kanade | Farneback |
|------|-------------|----------|
| **輸出** | 稀疏光流 (特徵點) | 稠密光流 (所有像素) |
| **速度** | 快 | 較慢 |
| **應用** | 物體追蹤 | 運動場分析、視頻穩定 |
| **記憶體** | 低 | 高 |
| **可視化** | 追蹤軌跡 | 運動向量場 |

## 3-2: OpenCV 實現

### cv2.calcOpticalFlowFarneback() 函數

```python
flow = cv2.calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, 
                                    winsize, iterations, poly_n, poly_sigma, flags)
```

> **參數說明**:
> * `prev`: 前一幀灰度影像
> * `next`: 當前幀灰度影像
> * `flow`: 初始光流 (可為 None)
> * `pyr_scale`: 金字塔縮放比例 (0.5 表示每層縮小一半)
> * `levels`: 金字塔層數
> * `winsize`: 平均窗口大小
> * `iterations`: 每層迭代次數
> * `poly_n`: 多項式展開的鄰域大小 (5 或 7)
> * `poly_sigma`: 高斯標準差 (通常 1.1 或 1.5)
> * `flags`: 操作標誌
>
> **返回值**:
> * `flow`: 光流場 (H × W × 2)，包含每個像素的 (dx, dy)

## 3-3: Farneback 稠密光流實作

In [ ]:
# Load video
video_path = '../assets/videos/car_chase_01.mp4'

if os.path.exists(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    
    # Resize for faster computation
    frame1 = cv2.resize(frame1, (640, 360))
else:
    # Use synthetic data
    frame1 = frames[0]
    cap = None

prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Parameters for Farneback optical flow
farneback_params = dict(
    pyr_scale=0.5,      # Pyramid scale
    levels=3,           # Number of pyramid levels
    winsize=15,         # Averaging window size
    iterations=3,       # Iterations at each pyramid level
    poly_n=5,           # Size of pixel neighborhood (5 or 7)
    poly_sigma=1.2,     # Gaussian sigma for polynomial expansion
    flags=0             # Operation flags
)

print('\n✅ Farneback 稠密光流設置:')
print('='*60)
print(f'影像尺寸: {frame1.shape[:2]}')
print(f'金字塔層數: {farneback_params["levels"]}')
print(f'窗口大小: {farneback_params["winsize"]}')
print(f'多項式鄰域: {farneback_params["poly_n"]}')

In [ ]:
import time

# Process multiple frames
flow_frames = []
num_frames = 20

# Reset video
if cap is not None:
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret, prev_frame = cap.read()
    prev_frame = cv2.resize(prev_frame, (640, 360))
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_idx = 0
else:
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
    frame_idx = 0

computation_times = []

for i in range(num_frames):
    # Read next frame
    if cap is not None:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (640, 360))
    else:
        if frame_idx + 1 >= len(frames):
            break
        frame = frames[frame_idx + 1]
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate dense optical flow
    start_time = time.time()
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, **farneback_params)
    computation_times.append(time.time() - start_time)
    
    # Visualize flow
    # 1. HSV representation
    flow_hsv = draw_hsv_flow(flow)
    
    # 2. Arrow representation
    flow_arrows = draw_flow(frame, flow, step=16)
    
    # 3. Magnitude visualization
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    mag_norm = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    mag_vis = cv2.applyColorMap(mag_norm.astype(np.uint8), cv2.COLORMAP_JET)
    
    flow_frames.append({
        'original': frame,
        'hsv': flow_hsv,
        'arrows': flow_arrows,
        'magnitude': mag_vis,
        'flow': flow
    })
    
    # Update for next iteration
    prev_gray = gray
    frame_idx += 1

if cap is not None:
    cap.release()

print(f'\n✅ Farneback 光流計算完成')
print('='*60)
print(f'處理幀數: {len(flow_frames)}')
print(f'平均計算時間: {np.mean(computation_times)*1000:.2f} ms')
print(f'處理速度: {1/np.mean(computation_times):.1f} FPS')

In [ ]:
# Display Farneback flow results
if len(flow_frames) > 0:
    # Select representative frames
    display_idx = len(flow_frames) // 2
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Original frame
    axes[0, 0].imshow(cv2.cvtColor(flow_frames[display_idx]['original'], cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('Original Frame', fontsize=14, fontweight='bold')
    axes[0, 0].axis('off')
    
    # HSV flow
    axes[0, 1].imshow(cv2.cvtColor(flow_frames[display_idx]['hsv'], cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title('Dense Flow (HSV: Hue=Direction, Value=Magnitude)', 
                        fontsize=14, fontweight='bold')
    axes[0, 1].axis('off')
    
    # Arrow visualization
    axes[1, 0].imshow(cv2.cvtColor(flow_frames[display_idx]['arrows'], cv2.COLOR_BGR2RGB))
    axes[1, 0].set_title('Flow Vectors (Arrows)', fontsize=14, fontweight='bold')
    axes[1, 0].axis('off')
    
    # Magnitude
    axes[1, 1].imshow(cv2.cvtColor(flow_frames[display_idx]['magnitude'], cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title('Flow Magnitude (Speed)', fontsize=14, fontweight='bold')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Flow statistics
    flow_data = flow_frames[display_idx]['flow']
    mag, ang = cv2.cartToPolar(flow_data[..., 0], flow_data[..., 1])
    
    print('\n📊 光流統計分析:')
    print('='*60)
    print(f'平均位移量: {np.mean(mag):.3f} pixels')
    print(f'最大位移量: {np.max(mag):.3f} pixels')
    print(f'運動像素比例: {np.sum(mag > 1.0) / mag.size * 100:.1f}%')
    print(f'主要運動方向: {np.degrees(np.mean(ang[mag > 1.0])):.1f}°')

### Farneback 參數調整對比

In [ ]:
# Compare different parameter settings
if len(flow_frames) > 0:
    # Get two consecutive frames
    gray1 = cv2.cvtColor(flow_frames[0]['original'], cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(flow_frames[5]['original'], cv2.COLOR_BGR2GRAY)
    
    # Test different parameters
    param_sets = [
        {'levels': 2, 'winsize': 10, 'name': 'Fast (Low Quality)'},
        {'levels': 3, 'winsize': 15, 'name': 'Balanced [Default]'},
        {'levels': 4, 'winsize': 20, 'name': 'High Quality (Slow)'},
    ]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    print('\n📊 Farneback 參數對比:')
    print('='*60)
    
    for idx, params in enumerate(param_sets):
        # Calculate flow with different parameters
        test_params = farneback_params.copy()
        test_params['levels'] = params['levels']
        test_params['winsize'] = params['winsize']
        
        start = time.time()
        flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, **test_params)
        elapsed = (time.time() - start) * 1000
        
        # Visualize
        flow_vis = draw_hsv_flow(flow)
        
        axes[idx].imshow(cv2.cvtColor(flow_vis, cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f"{params['name']}\n{elapsed:.1f} ms", 
                           fontsize=12, fontweight='bold')
        axes[idx].axis('off')
        
        print(f"{params['name']:20} | Levels: {params['levels']} | "
              f"WinSize: {params['winsize']} | Time: {elapsed:.1f} ms")
    
    plt.tight_layout()
    plt.show()
    
    print('\n💡 參數調整建議:')
    print('   - levels 高: 可處理更大的運動，但計算慢')
    print('   - winsize 大: 光流更平滑，但細節較少')
    print('   - poly_n (5/7): 5 較快，7 較準確')
    print('='*60)

<div style="page-break-after: always"></div>

# 4. 稀疏光流 vs 稠密光流比較

## 4-1: 視覺化比較

In [ ]:
# Side-by-side comparison of sparse and dense optical flow

# Load or create test frames
video_path = '../assets/videos/car_chase_01.mp4'

if os.path.exists(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    frame1 = cv2.resize(frame1, (640, 360))
    
    for _ in range(5):
        ret, frame2 = cap.read()
    frame2 = cv2.resize(frame2, (640, 360))
    cap.release()
else:
    frame1 = frames[0]
    frame2 = frames[5]

gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

# 1. Lucas-Kanade (Sparse)
p0 = cv2.goodFeaturesToTrack(gray1, maxCorners=200, qualityLevel=0.3, 
                             minDistance=7, blockSize=7)

lk_params = dict(winSize=(15, 15), maxLevel=2,
                criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

start_lk = time.time()
p1, st, err = cv2.calcOpticalFlowPyrLK(gray1, gray2, p0, None, **lk_params)
time_lk = (time.time() - start_lk) * 1000

# Visualize sparse flow
frame_lk = frame2.copy()
if p1 is not None:
    good_new = p1[st == 1]
    good_old = p0[st == 1]
    
    for new, old in zip(good_new, good_old):
        a, b = new.ravel()
        c, d = old.ravel()
        frame_lk = cv2.arrowedLine(frame_lk, (int(c), int(d)), (int(a), int(b)),
                                   (0, 255, 0), 2, tipLength=0.3)
        frame_lk = cv2.circle(frame_lk, (int(a), int(b)), 3, (0, 0, 255), -1)

# 2. Farneback (Dense)
start_fb = time.time()
flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
time_fb = (time.time() - start_fb) * 1000

frame_fb = draw_hsv_flow(flow)

# Display comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].imshow(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Frame t', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Frame t+1', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(cv2.cvtColor(frame_lk, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title(f'Lucas-Kanade (Sparse)\n'
                    f'{len(good_new)} points tracked | {time_lk:.1f} ms',
                    fontsize=14, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(frame_fb, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title(f'Farneback (Dense)\n'
                    f'{flow.shape[0]*flow.shape[1]} vectors | {time_fb:.1f} ms',
                    fontsize=14, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print('\n📊 稀疏 vs 稠密光流比較:')
print('='*60)
print(f'Lucas-Kanade (稀疏):')
print(f'  - 追蹤點數: {len(good_new)}')
print(f'  - 計算時間: {time_lk:.2f} ms')
print(f'  - 記憶體: 低 (只儲存特徵點)')
print(f'  - 適用: 物體追蹤、相機運動估計')
print()
print(f'Farneback (稠密):')
print(f'  - 向量數量: {flow.shape[0] * flow.shape[1]:,}')
print(f'  - 計算時間: {time_fb:.2f} ms')
print(f'  - 記憶體: 高 (儲存所有像素)')
print(f'  - 適用: 運動分析、視頻穩定、視覺效果')
print()
print(f'速度比: Lucas-Kanade 快 {time_fb/time_lk:.1f}x')
print('='*60)

## 4-2: 特性對比表

| 特性 | Lucas-Kanade (稀疏) | Farneback (稠密) |
|------|-------------------|----------------|
| **計算範圍** | 選定的特徵點 | 所有像素 |
| **輸出** | N個向量 (N<1000) | H×W 個向量 (>100,000) |
| **速度** | ⚡⚡⚡ 快 | ⭐⭐ 較慢 |
| **記憶體** | 低 | 高 |
| **準確度** | 高 (特徵點處) | 中等 (全域) |
| **適用場景** | 物體追蹤、SLAM | 運動分析、視頻穩定 |
| **實時性** | ✅ 適合 | ⚠️ 需要優化 |
| **魯棒性** | 依賴特徵檢測 | 對紋理敏感 |

## 4-3: 應用場景決策樹

```
選擇光流方法?
├─ 需要追蹤特定物體?
│   └─ 是 → Lucas-Kanade (稀疏光流)
│
├─ 需要分析整體運動模式?
│   └─ 是 → Farneback (稠密光流)
│
├─ 需要實時處理 (>30 FPS)?
│   └─ 是 → Lucas-Kanade
│
├─ 需要視頻穩定或特效?
│   └─> 是 → Farneback
│
└─ 高精度物體追蹤?
    └─ 使用 Lucas-Kanade + 卡爾曼濾波
```

<div style="page-break-after: always"></div>

# 5. 多目標追蹤基礎

## 5-1: 多目標追蹤概述

> **多目標追蹤 (Multi-Object Tracking, MOT)** 是在視頻序列中同時追蹤多個目標物體的技術。

### 核心挑戰

```
多目標追蹤的挑戰:

1. 數據關聯 (Data Association)
   └─> 將檢測結果與已有軌跡配對

2. 遮擋處理 (Occlusion Handling)
   └─> 物體被遮擋時如何維持追蹤

3. 目標外觀變化
   └─> 光照、視角、姿態變化

4. 新目標進入與離開
   └─> 動態管理追蹤目標
```

### 追蹤流程

```
MOT 典型流程:

Frame t
  ↓
1. 物體檢測 (Detection)
  └─> 找到所有候選目標
  ↓
2. 特徵提取 (Feature Extraction)
  └─> 計算目標特徵 (位置、外觀、運動)
  ↓
3. 數據關聯 (Data Association)
  └─> 匹配檢測結果與已有軌跡
  ↓
4. 狀態更新 (State Update)
  └─> 更新軌跡狀態 (卡爾曼濾波)
  ↓
5. 軌跡管理 (Track Management)
  └─> 創建、維護、刪除軌跡
```

## 5-2: 基於光流的多目標追蹤

In [ ]:
class SimpleTracker:
    """
    Simple multi-object tracker using optical flow
    """
    def __init__(self, max_age=30, min_hits=3):
        """
        Args:
            max_age: Maximum frames to keep lost tracks
            min_hits: Minimum detections before confirmed track
        """
        self.tracks = []  # List of active tracks
        self.next_id = 0
        self.max_age = max_age
        self.min_hits = min_hits
        
        # Lucas-Kanade parameters
        self.lk_params = dict(
            winSize=(21, 21),
            maxLevel=3,
            criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
        )
    
    def update(self, frame, detections):
        """
        Update tracker with new frame and detections
        
        Args:
            frame: Current frame (grayscale)
            detections: List of detected bounding boxes [(x, y, w, h), ...]
        
        Returns:
            tracks: List of active tracks with IDs
        """
        # Initialize tracks from detections if first frame
        if len(self.tracks) == 0:
            for det in detections:
                self.tracks.append({
                    'id': self.next_id,
                    'bbox': det,
                    'age': 0,
                    'hits': 1,
                    'points': self._get_feature_points(frame, det),
                    'prev_frame': frame.copy()
                })
                self.next_id += 1
            return self.tracks
        
        # Track existing points using optical flow
        for track in self.tracks:
            if track['points'] is not None and len(track['points']) > 0:
                # Calculate optical flow
                new_points, status, _ = cv2.calcOpticalFlowPyrLK(
                    track['prev_frame'], frame, track['points'], None, **self.lk_params
                )
                
                if new_points is not None:
                    good_new = new_points[status == 1]
                    
                    if len(good_new) > 0:
                        # Update bounding box based on tracked points
                        x_coords = good_new[:, 0]
                        y_coords = good_new[:, 1]
                        
                        x = int(np.min(x_coords))
                        y = int(np.min(y_coords))
                        w = int(np.max(x_coords) - x)
                        h = int(np.max(y_coords) - y)
                        
                        track['bbox'] = (x, y, w, h)
                        track['points'] = good_new.reshape(-1, 1, 2)
                        track['age'] = 0  # Reset age
                    else:
                        track['age'] += 1
                else:
                    track['age'] += 1
            else:
                track['age'] += 1
            
            track['prev_frame'] = frame.copy()
        
        # Remove old tracks
        self.tracks = [t for t in self.tracks if t['age'] < self.max_age]
        
        # Add new detections (simple: add if far from existing tracks)
        for det in detections:
            is_new = True
            for track in self.tracks:
                if self._bbox_iou(det, track['bbox']) > 0.3:
                    is_new = False
                    # Refresh feature points
                    track['points'] = self._get_feature_points(frame, det)
                    track['hits'] += 1
                    break
            
            if is_new:
                self.tracks.append({
                    'id': self.next_id,
                    'bbox': det,
                    'age': 0,
                    'hits': 1,
                    'points': self._get_feature_points(frame, det),
                    'prev_frame': frame.copy()
                })
                self.next_id += 1
        
        return [t for t in self.tracks if t['hits'] >= self.min_hits]
    
    def _get_feature_points(self, frame, bbox):
        """Extract feature points within bounding box"""
        x, y, w, h = bbox
        
        # Ensure bbox is within frame
        x = max(0, x)
        y = max(0, y)
        w = min(w, frame.shape[1] - x)
        h = min(h, frame.shape[0] - y)
        
        if w <= 0 or h <= 0:
            return None
        
        roi = frame[y:y+h, x:x+w]
        
        if roi.size == 0:
            return None
        
        # Detect features in ROI
        points = cv2.goodFeaturesToTrack(roi, maxCorners=20, qualityLevel=0.01,
                                        minDistance=5, blockSize=7)
        
        if points is not None:
            # Offset points to global coordinates
            points[:, :, 0] += x
            points[:, :, 1] += y
            return points
        return None
    
    def _bbox_iou(self, bbox1, bbox2):
        """Calculate IoU between two bounding boxes"""
        x1, y1, w1, h1 = bbox1
        x2, y2, w2, h2 = bbox2
        
        # Calculate intersection
        xi1 = max(x1, x2)
        yi1 = max(y1, y2)
        xi2 = min(x1 + w1, x2 + w2)
        yi2 = min(y1 + h1, y2 + h2)
        
        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
        
        # Calculate union
        box1_area = w1 * h1
        box2_area = w2 * h2
        union_area = box1_area + box2_area - inter_area
        
        return inter_area / union_area if union_area > 0 else 0

print('✅ SimpleTracker class defined')

### 多目標追蹤示範

In [ ]:
# Demonstrate multi-object tracking

# For demonstration, we'll create synthetic detections
# In real applications, use object detection models (YOLO, SSD, etc.)

def simulate_detections(frame_idx, num_frames):
    """
    Simulate object detections for demonstration
    In practice, use real object detector
    """
    detections = []
    
    # Simulate two moving objects
    # Object 1: moving right
    x1 = 50 + frame_idx * 10
    y1 = 100
    if x1 < 550:
        detections.append((x1, y1, 80, 80))
    
    # Object 2: moving diagonally
    x2 = 500 - frame_idx * 8
    y2 = 50 + frame_idx * 5
    if x2 > 50 and y2 < 300:
        detections.append((x2, y2, 60, 60))
    
    # Object 3: appears later
    if frame_idx > 15:
        x3 = 300
        y3 = 200 + (frame_idx - 15) * 3
        if y3 < 320:
            detections.append((x3, y3, 70, 70))
    
    return detections


# Create synthetic video for tracking
tracker = SimpleTracker(max_age=10, min_hits=2)
tracking_frames = []
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]

num_frames = 40

for frame_idx in range(num_frames):
    # Create frame
    frame = np.ones((400, 640, 3), dtype=np.uint8) * 255
    
    # Get simulated detections
    detections = simulate_detections(frame_idx, num_frames)
    
    # Draw detections (dashed rectangles)
    for det in detections:
        x, y, w, h = det
        # Draw dashed rectangle
        cv2.rectangle(frame, (x, y), (x+w, y+h), (128, 128, 128), 1, cv2.LINE_AA)
        # Fill with semi-transparent color
        overlay = frame.copy()
        cv2.rectangle(overlay, (x, y), (x+w, y+h), (200, 200, 200), -1)
        cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
    
    # Convert to grayscale for tracking
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Update tracker
    tracks = tracker.update(gray, detections)
    
    # Draw tracks
    for track in tracks:
        track_id = track['id']
        x, y, w, h = track['bbox']
        color = colors[track_id % len(colors)]
        
        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
        
        # Draw ID
        cv2.putText(frame, f'ID: {track_id}', (x, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        # Draw tracked points
        if track['points'] is not None:
            for pt in track['points']:
                px, py = pt.ravel()
                cv2.circle(frame, (int(px), int(py)), 3, color, -1)
    
    # Add frame info
    cv2.putText(frame, f'Frame: {frame_idx+1}/{num_frames}', (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    cv2.putText(frame, f'Active Tracks: {len(tracks)}', (10, 60),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    
    tracking_frames.append(frame)

# Display tracking results
if len(tracking_frames) > 0:
    display_indices = [0, 10, 20, 30, 39]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    for idx, i in enumerate(display_indices):
        axes[idx].imshow(cv2.cvtColor(tracking_frames[i], cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f'Frame {i+1}', fontsize=14, fontweight='bold')
        axes[idx].axis('off')
    
    # Legend
    axes[5].text(0.1, 0.9, 'Legend:', fontsize=14, fontweight='bold', 
                transform=axes[5].transAxes)
    axes[5].text(0.1, 0.7, '■ Gray dashed box: Detection', fontsize=12,
                transform=axes[5].transAxes, color='gray')
    axes[5].text(0.1, 0.5, '■ Colored solid box: Tracked object', fontsize=12,
                transform=axes[5].transAxes, color='blue')
    axes[5].text(0.1, 0.3, '● Points: Tracked features', fontsize=12,
                transform=axes[5].transAxes)
    axes[5].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print('\n✅ 多目標追蹤演示完成')
    print('='*60)
    print('觀察:')
    print('  1. 每個目標被分配唯一 ID')
    print('  2. 使用光流追蹤特徵點')
    print('  3. 新目標出現時自動分配新 ID')
    print('  4. 失去追蹤的目標會在一段時間後被移除')

<div style="page-break-after: always"></div>

# 6. 卡爾曼濾波器基礎 (進階)

## 6-1: 卡爾曼濾波器概述

> **卡爾曼濾波器 (Kalman Filter)** 是一種遞迴貝葉斯濾波器，用於估計動態系統的狀態。

### 為什麼需要卡爾曼濾波？

```
物體追蹤中的問題:

1. 測量噪聲
   └─> 檢測器不完美，位置有誤差

2. 暫時遮擋
   └─> 物體被遮擋時無法檢測

3. 運動預測
   └─> 需要預測下一幀的位置

卡爾曼濾波的解決方案:
- 融合預測值與測量值
- 估計最優狀態
- 提供不確定性度量
```

### 卡爾曼濾波器的兩個步驟

> **1. 預測 (Prediction)**:
>
> $$\hat{x}_{k|k-1} = F_k \hat{x}_{k-1|k-1} + B_k u_k$$
> $$P_{k|k-1} = F_k P_{k-1|k-1} F_k^T + Q_k$$
>
> **2. 更新 (Update)**:
>
> $$K_k = P_{k|k-1} H_k^T (H_k P_{k|k-1} H_k^T + R_k)^{-1}$$
> $$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k (z_k - H_k \hat{x}_{k|k-1})$$
> $$P_{k|k} = (I - K_k H_k) P_{k|k-1}$$

### 矩陣說明

| 矩陣 | 名稱 | 說明 |
|------|------|------|
| $F_k$ | 狀態轉移矩陣 | 描述狀態如何隨時間變化 |
| $H_k$ | 觀測矩陣 | 將狀態映射到測量空間 |
| $Q_k$ | 過程噪聲協方差 | 模型不確定性 |
| $R_k$ | 測量噪聲協方差 | 測量不確定性 |
| $K_k$ | 卡爾曼增益 | 決定信任預測還是測量 |

## 6-2: 簡單的 1D 追蹤範例

In [ ]:
class SimpleKalmanFilter:
    """
    Simple 1D Kalman Filter for demonstration
    State: [position, velocity]
    """
    def __init__(self, process_noise=0.1, measurement_noise=1.0):
        # State: [position, velocity]
        self.x = np.array([[0.0], [0.0]])  # Initial state
        
        # State covariance
        self.P = np.eye(2) * 1000  # High uncertainty initially
        
        # State transition matrix (constant velocity model)
        dt = 1.0  # Time step
        self.F = np.array([[1, dt],
                          [0, 1]])
        
        # Observation matrix (we only measure position)
        self.H = np.array([[1, 0]])
        
        # Process noise covariance
        self.Q = np.eye(2) * process_noise
        
        # Measurement noise covariance
        self.R = np.array([[measurement_noise]])
    
    def predict(self):
        """
        Prediction step
        """
        # Predict state
        self.x = self.F @ self.x
        
        # Predict covariance
        self.P = self.F @ self.P @ self.F.T + self.Q
        
        return self.x[0, 0]
    
    def update(self, measurement):
        """
        Update step with measurement
        """
        # Measurement residual
        y = np.array([[measurement]]) - self.H @ self.x
        
        # Residual covariance
        S = self.H @ self.P @ self.H.T + self.R
        
        # Kalman gain
        K = self.P @ self.H.T @ np.linalg.inv(S)
        
        # Update state
        self.x = self.x + K @ y
        
        # Update covariance
        I = np.eye(2)
        self.P = (I - K @ self.H) @ self.P
        
        return self.x[0, 0]

# Demonstrate Kalman Filter
print('\n📊 卡爾曼濾波器演示 (1D 追蹤):')
print('='*60)

# Generate synthetic data
np.random.seed(42)
true_positions = np.linspace(0, 100, 50) + np.sin(np.linspace(0, 4*np.pi, 50)) * 5
measurements = true_positions + np.random.randn(50) * 3  # Add noise

# Apply Kalman filter
kf = SimpleKalmanFilter(process_noise=0.1, measurement_noise=3.0)
filtered_positions = []
predicted_positions = []

for measurement in measurements:
    # Predict
    prediction = kf.predict()
    predicted_positions.append(prediction)
    
    # Update with measurement
    filtered = kf.update(measurement)
    filtered_positions.append(filtered)

# Visualize
plt.figure(figsize=(14, 6))

plt.plot(true_positions, 'g-', linewidth=2, label='True Position', alpha=0.7)
plt.plot(measurements, 'rx', markersize=8, label='Noisy Measurements', alpha=0.5)
plt.plot(filtered_positions, 'b-', linewidth=2, label='Kalman Filter Output')
plt.plot(predicted_positions, 'y--', linewidth=1.5, label='Predictions', alpha=0.7)

plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Position', fontsize=12)
plt.title('Kalman Filter: Smoothing Noisy Measurements', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate errors
measurement_error = np.mean(np.abs(measurements - true_positions))
filtered_error = np.mean(np.abs(np.array(filtered_positions) - true_positions))

print(f'平均測量誤差: {measurement_error:.2f}')
print(f'平均濾波誤差: {filtered_error:.2f}')
print(f'誤差降低: {(1 - filtered_error/measurement_error)*100:.1f}%')
print('\n💡 卡爾曼濾波器的效果:')
print('   - 平滑噪聲測量')
   '   - 預測未來位置')
print('   - 減少誤差')
print('='*60)

## 6-3: 卡爾曼濾波器在物體追蹤中的應用

### 常用狀態模型

```python
# 1. 恆速模型 (Constant Velocity)
# State: [x, y, vx, vy]
state = [position_x, position_y, velocity_x, velocity_y]

# 2. 恆加速度模型 (Constant Acceleration)
# State: [x, y, vx, vy, ax, ay]
state = [position_x, position_y, velocity_x, velocity_y, accel_x, accel_y]

# 3. 中心點+尺寸模型 (Center + Scale)
# State: [cx, cy, w, h, vx, vy, vw, vh]
state = [center_x, center_y, width, height, v_x, v_y, v_w, v_h]
```

### 實際應用案例

| 應用 | 狀態模型 | 特點 |
|------|---------|------|
| **行人追蹤** | 恆速模型 | 運動相對平滑 |
| **車輛追蹤** | 恆速/恆加速 | 需考慮加速度 |
| **無人機追蹤** | 中心點+尺寸 | 尺度變化大 |
| **運動分析** | 關節點模型 | 多個關鍵點 |

### 進階: SORT / DeepSORT

> **SORT** (Simple Online and Realtime Tracking):
> * 結合卡爾曼濾波器與匈牙利算法
> * 用於數據關聯
> * 適合實時追蹤
>
> **DeepSORT**:
> * SORT + 深度外觀特徵
> * 使用 CNN 提取外觀特徵
> * 更強的遮擋處理能力

<div style="page-break-after: always"></div>

# 7. 實戰練習

## 練習 1: 視頻特徵點追蹤

In [ ]:
# TODO: 實現視頻特徵點追蹤
# 任務: 使用 Lucas-Kanade 追蹤視頻中的特徵點，繪製運動軌跡

print('\n📝 練習 1: 視頻特徵點追蹤')
print('='*60)
print('任務:')
print('  1. 讀取視頻文件')
print('  2. 使用 Shi-Tomasi 檢測第一幀的特徵點')
print('  3. 使用 Lucas-Kanade 追蹤特徵點')
print('  4. 繪製追蹤軌跡')
print('  5. 統計追蹤成功率')
print('\n提示:')
print('  - 使用 cv2.goodFeaturesToTrack() 檢測特徵')
print('  - 使用 cv2.calcOpticalFlowPyrLK() 追蹤')
print('  - 根據 status 過濾失敗的追蹤')
print('='*60)

# Solution framework
def track_video_features(video_path, max_frames=50):
    """
    Track features in video using Lucas-Kanade
    """
    # TODO: Implement feature tracking
    # Your code here
    pass

# Uncomment to test
# track_video_features('../assets/videos/car_chase_01.mp4')

## 練習 2: 運動向量視覺化

In [ ]:
# TODO: 實現運動向量視覺化
# 任務: 使用 Farneback 計算稠密光流，並進行多種視覺化

print('\n📝 練習 2: 運動向量視覺化')
print('='*60)
print('任務:')
print('  1. 讀取連續的兩幀影像')
print('  2. 使用 Farneback 計算稠密光流')
print('  3. 實現三種視覺化方式:')
print('     a) HSV 顏色編碼')
print('     b) 箭頭向量場')
print('     c) 運動幅度熱力圖')
print('  4. 分析運動統計 (平均速度、方向分佈)')
print('\n提示:')
print('  - 使用 cv2.calcOpticalFlowFarneback()')
print('  - 使用 cv2.cartToPolar() 轉換為極座標')
print('  - 參考本模組的 draw_flow() 和 draw_hsv_flow() 函數')
print('='*60)

# Solution framework
def visualize_motion_vectors(frame1, frame2):
    """
    Visualize optical flow in multiple ways
    """
    # TODO: Implement motion vector visualization
    # Your code here
    pass

# Uncomment to test
# visualize_motion_vectors(frame1, frame2)

## 練習 3: 簡易物體追蹤系統 (挑戰)

In [ ]:
# TODO: 實現簡易物體追蹤系統
# 任務: 整合特徵檢測、光流追蹤和卡爾曼濾波

print('\n📝 練習 3: 簡易物體追蹤系統 (挑戰)')
print('='*60)
print('任務:')
print('  1. 讓用戶在第一幀選擇追蹤目標 (ROI)')
print('  2. 使用光流法追蹤目標')
print('  3. 使用卡爾曼濾波器預測位置')
print('  4. 處理遮擋情況 (依賴預測)')
print('  5. 顯示追蹤結果和置信度')
print('\n進階挑戰:')
print('  - 支持多目標追蹤')
print('  - 自動重新初始化丟失的追蹤')
print('  - 添加追蹤質量評估')
print('='*60)

# Solution framework
class ObjectTracker:
    def __init__(self):
        # TODO: Initialize tracker components
        pass
    
    def initialize(self, frame, bbox):
        # TODO: Initialize tracking with first frame and bbox
        pass
    
    def update(self, frame):
        # TODO: Update tracking with new frame
        pass

# Uncomment to test
# tracker = ObjectTracker()
# tracker.initialize(first_frame, bbox)
# for frame in video_frames:
#     result = tracker.update(frame)

<div style="page-break-after: always"></div>

# 8. 總結與延伸

## 8-1: 核心要點回顧

### 光流法

> **Lucas-Kanade (稀疏光流)**:
> * 📌 追蹤特定特徵點
> * 📌 使用局部鄰域假設求解光流
> * 📌 金字塔結構處理大位移
> * 📌 適合物體追蹤、視覺測程
>
> **Farneback (稠密光流)**:
> * 📌 計算所有像素的光流
> * 📌 基於多項式展開
> * 📌 適合運動分析、視頻穩定
> * 📌 計算量大，需要優化

### 物體追蹤

> **多目標追蹤流程**:
> 1. 物體檢測 (Detection)
> 2. 特徵提取 (Feature Extraction)
> 3. 數據關聯 (Data Association)
> 4. 狀態更新 (State Update)
> 5. 軌跡管理 (Track Management)

### 卡爾曼濾波器

> **兩步驟流程**:
> 1. 預測 (Prediction): 根據運動模型預測狀態
> 2. 更新 (Update): 融合測量值校正預測
>
> **優勢**:
> * 平滑噪聲測量
> * 預測未來位置
> * 處理暫時遮擋

## 8-2: 性能比較總結

| 方法 | 速度 | 準確度 | 魯棒性 | 適用場景 |
|------|------|--------|--------|----------|
| **Lucas-Kanade** | ⚡⚡⚡ | ⭐⭐⭐⭐ | ⭐⭐⭐ | 物體追蹤、SLAM |
| **Farneback** | ⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | 運動分析、視頻穩定 |
| **LK + Kalman** | ⚡⚡ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | 高精度追蹤 |
| **DeepSORT** | ⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | 多目標追蹤 |

## 8-3: 應用場景決策樹

```
選擇追蹤方法?
├─ 單一目標追蹤?
│   ├─ 需要實時 (>30 FPS)?
│   │   └─ Lucas-Kanade
│   └─ 需要高精度?
│       └─ Lucas-Kanade + Kalman Filter
│
├─ 多目標追蹤?
│   ├─ 簡單場景?
│   │   └─ SORT (Kalman + Hungarian)
│   └─ 複雜場景 (遮擋多)?
│       └─ DeepSORT (SORT + Deep Features)
│
├─ 運動場分析?
│   └─ Farneback Dense Flow
│
└─ 視頻穩定?
    └─ Farneback + 平滑處理
```

## 8-4: 實用建議

### 參數調整技巧

```python
# Lucas-Kanade 參數
lk_params = dict(
    winSize=(21, 21),      # 大位移用大窗口
    maxLevel=3,            # 金字塔層數，處理大運動
    criteria=(...)         # 迭代終止條件
)

# Farneback 參數
fb_params = dict(
    pyr_scale=0.5,         # 金字塔縮放
    levels=3,              # 層數越多越慢但更準確
    winsize=15,            # 窗口越大越平滑
    iterations=3,          # 迭代次數
    poly_n=5,              # 5 或 7
    poly_sigma=1.2         # 1.1 ~ 1.5
)
```

### 性能優化建議

> 1. **影像預處理**:
>    * 降低解析度 (640x480 通常足夠)
>    * 使用 ROI 限制處理區域
>    * 高斯模糊去噪
>
> 2. **特徵點管理**:
>    * 定期重新檢測特徵點
>    * 移除品質差的追蹤點
>    * 維持適當的特徵點密度
>
> 3. **多線程加速**:
>    * 檢測和追蹤並行處理
>    * 使用 GPU 加速 (CUDA)

## 8-5: 延伸學習

### 進階主題

> * **深度學習光流**: FlowNet, PWC-Net, RAFT
> * **3D 運動估計**: 場景流 (Scene Flow)
> * **擴展卡爾曼濾波**: EKF, UKF
> * **粒子濾波器**: 非線性追蹤
> * **視覺慣性測程**: VIO (Visual-Inertial Odometry)

### 下一步學習

- [x] 角點檢測 (4.1.1) ✅
- [x] 特徵描述子 (4.1.2) ✅
- [x] 物體追蹤 (4.1.3) ✅ ← **你在這裡**
- [ ] 深度學習物體檢測 (YOLO, SSD)
- [ ] 實時多目標追蹤專案
- [ ] SLAM (同步定位與地圖構建)

## 8-6: 參考資源

### 論文

> * **Lucas-Kanade**:  
>   Lucas, B. D., & Kanade, T. (1981). "An iterative image registration technique."
>
> * **Pyramidal Lucas-Kanade**:  
>   Bouguet, J. Y. (2001). "Pyramidal implementation of the Lucas Kanade feature tracker."
>
> * **Farneback**:  
>   Farnebäck, G. (2003). "Two-frame motion estimation based on polynomial expansion."
>
> * **Kalman Filter**:  
>   Kalman, R. E. (1960). "A new approach to linear filtering and prediction problems."
>
> * **SORT**:  
>   Bewley et al. (2016). "Simple Online and Realtime Tracking."
>
> * **DeepSORT**:  
>   Wojke et al. (2017). "Simple Online and Realtime Tracking with a Deep Association Metric."

### 線上資源

> * OpenCV Optical Flow Tutorial: https://docs.opencv.org/master/d4/dee/tutorial_optical_flow.html
> * Kalman Filter Explained: https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/
> * Multiple Object Tracking: https://motchallenge.net/

---

## 🎯 學習成果檢查清單

完成本模組後，你應該能夠:

- [ ] 理解光流法的基本原理和假設
- [ ] 實現 Lucas-Kanade 稀疏光流追蹤
- [ ] 實現 Farneback 稠密光流分析
- [ ] 比較稀疏光流和稠密光流的差異
- [ ] 建立基本的多目標追蹤系統
- [ ] 理解卡爾曼濾波器的工作原理
- [ ] 將卡爾曼濾波器應用於物體追蹤
- [ ] 根據應用場景選擇適當的追蹤方法
- [ ] 調整參數以優化追蹤性能
- [ ] 處理追蹤中的常見問題 (遮擋、漂移)

---

**模組完成！繼續學習機器學習模組 (05_machine_learning)，探索深度學習在計算機視覺中的應用。**